In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers

%matplotlib inline
import cv2
import PIL

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(
    "flower_photos", origin=dataset_url, untar=True, cache_dir="."
)

In [ ]:
import pathlib

data_dir = pathlib.Path("datasets/flower_photos/flower_photos")
data_dir

In [ ]:
image_count = len(list(data_dir.glob("*/*.jpg")))
image_count

In [ ]:
roses = list(data_dir.glob("roses/*"))
PIL.Image.open(str(roses[1]))

In [ ]:
tulips = list(data_dir.glob("tulips/*"))

In [ ]:
daisy = list(data_dir.glob("daisy/*"))
dandelion = list(data_dir.glob("dandelion/*"))
sunflowers = list(data_dir.glob("sunflowers/*"))

In [ ]:
flowers_image_dict = {
    "roses": roses,
    "daisy": daisy,
    "dandelion": dandelion,
    "sunflowers": sunflowers,
    "tulips": tulips,
}

In [ ]:
flowers_labels_dict = {
    "roses": 0,
    "daisy": 1,
    "dandelion": 2,
    "sunflowers": 3,
    "tulips": 4,
}

In [ ]:
img = cv2.imread(flowers_image_dict["roses"][0])
img.shape

In [ ]:
cv2.resize(img, (180, 180), interpolation=cv2.INTER_LANCZOS4).shape

In [ ]:
X, y = [], []
for flower_name, images in flowers_image_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, (180, 180), interpolation=cv2.INTER_LANCZOS4)
        X.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

In [ ]:
len(X_train), len(X_test)

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
model = keras.Sequential(
    [
        layers.Conv2D(16, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(5, activation="softmax"),
    ]
)

In [ ]:
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [ ]:
model.fit(X_train, y_train, epochs=10)

In [ ]:
model.evaluate(X_test, y_test)

In [73]:
data_augmentation = keras.Sequential(
    [
        layers.Input(shape=(180, 180, 3)),
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.4),
        layers.RandomZoom(0.2),
    ]
)

In [ ]:
plt.axis("off")
plt.imshow(X[0])

In [ ]:
plt.axis("off")
plt.imshow(data_augmentation(X)[0].numpy().astype("uint8"))

In [74]:
model = keras.Sequential(
    [
        data_augmentation,
        layers.Conv2D(16, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(5, activation="softmax"),
    ]
)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

history = model.fit(
    X_train,
    y_train,
    epochs=25,
    validation_data=(X_test, y_test),
)

Epoch 1/25
86/86 ━━━━━━━━━━━━━━━━━━━━ 11s 123ms/step - accuracy: 0.4008 - loss: 1.4439 - val_accuracy: 0.4444 - val_loss: 1.3057
Epoch 2/25
86/86 ━━━━━━━━━━━━━━━━━━━━ 10s 119ms/step - accuracy: 0.5265 - loss: 1.1303 - val_accuracy: 0.5806 - val_loss: 1.1092
Epoch 3/25
86/86 ━━━━━━━━━━━━━━━━━━━━ 10s 116ms/step - accuracy: 0.5868 - loss: 1.0330 - val_accuracy: 0.5861 - val_loss: 1.0864
Epoch 4/25
86/86 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - accuracy: 0.6195 - loss: 0.9627 - val_accuracy: 0.5817 - val_loss: 1.0954
Epoch 5/25
86/86 ━━━━━━━━━━━━━━━━━━━━ 12s 134ms/step - accuracy: 0.6424 - loss: 0.9159 - val_accuracy: 0.6264 - val_loss: 0.9612
Epoch 6/25
86/86 ━━━━━━━━━━━━━━━━━━━━ 10s 121ms/step - accuracy: 0.6632 - loss: 0.8752 - val_accuracy: 0.6296 - val_loss: 0.9386
Epoch 7/25
86/86 ━━━━━━━━━━━━━━━━━━━━ 10s 115ms/step - accuracy: 0.6686 - loss: 0.8529 - val_accuracy: 0.6242 - val_loss: 0.9691
Epoch 8/25
86/86 ━━━━━━━━━━━━━━━━━━━━ 10s 117ms/step - accuracy: 0.6675 - loss: 0.8351 - val_accu

In [75]:
model.evaluate(X_test, y_test)

29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7190 - loss: 0.7553


[0.755286455154419, 0.7189542651176453]